In [1]:
include("prepare.jl")
include("L1AC.jl")

L1AC! (generic function with 2 methods)

In [ ]:
total_steps = 15000

R0 = 10
β0 = - π / 2 * (3/4)
R_ref = R0
β_ref = - π / 2 * (3/4)
R_l1ac = R0

data = []

car = Car(state=State(x=-1., y=-11., ψ = 0., ẋ = 1., ẏ = 0., ψ̇ = 0.),tyre=Tyre(B=5, C=2, D=0.3))
β_controller = PID(Kp=4, Ki=0.001, Kd=60, sign=1)
R_controller = PID(Kp=30, Ki=0.001, Kd=5, sign=-1)
#R_controller = PID(Kp=100, Ki=0.8, Kd=100, sign=-1) 无前馈最佳参数
est = make_radius_estimator(10)
est_circles = []
β_refs = []
R_refs = []
l1ac = L1AC_Struct()

ω0 = 20
ω_ff = 0

est_circle = update!(est, car.state.x, car.state.y)

Cx = 0.
Cy = 0.

texture = "snow"

for i in 1:total_steps

    if texture == "snow"
        β_ff, δ_ff = β_ff_snow, δ_ff_snow
        car.tyre = Tyre(B=5, C=2, D=0.3)
        ω_ff = 32.27746
    else
        β_ff, δ_ff = β_ff_ice, δ_ff_ice
        car.tyre = Tyre(B=4, C=2, D=0.15)
        ω_ff = 24.79148
    end

    est_circle = update!(est, car.state.x, car.state.y)
    R = est_circle.r
    
    s = car.state
    
    β = atan(s.ẏ, s.ẋ) - s.ψ
    err = round_angle(β - β_ref)
    δ = clamp(control!(β_controller, err), -π / 8, π / 8)
    
    if i % 10 == 0
        
        angle1 = atan(Cy - s.y, Cx - s.x)
        angle2 = atan(s.ẏ, s.ẋ)
        dangle = round_angle(angle1 - angle2)
        dis = sqrt((s.x - Cx) ^ 2 + (s.y - Cy) ^ 2)
        dR = R0 / 2 * tanh(dis) * cos(dangle)

        R_ref = R0 + dR
        if i > 100
            R_l1ac = L1AC!(l1ac, convert(Float64,R_ref), est_circle.r, 0.10)
            err = R - R_l1ac
            ω0 = clamp(ω_ff + control!(R_controller, err), 15, 45)
        else
            err = R - R_ref
            ω0 = clamp(ω_ff + control!(R_controller, err), 15, 45)
        end
    end
    ω = ω0
    # apply control and save data
    u = Input(δ, ω, ω)
    data_point = DriftData(s, u, compute_extended_state(s, u, car.body, car.tyre), R)
    step!(car, u, dt=0.01)
    
    if i < total_steps / 3
        texture = "ice"
    elseif i < total_steps * 2 / 3
        texture = "ice"
    else
        texture = "ice"
    end

    push!(data,  data_point)
    push!(β_refs, β_ref)
    push!(R_refs, R_ref)
    push!(est_circles, est_circle)
end

In [ ]:
total_steps = 80000

R0 = 10
β0 = - π / 2 * (3/4)
R_ref = R0
β_ref = - π / 2 * (3/4)
R_l1ac = R0

data = []

car = Car(state=State(x=-16., y=-11., ψ = 0., ẋ = 1., ẏ = 0., ψ̇ = 0.),tyre=Tyre(B=5, C=2, D=0.3))
β_controller = PID(Kp=4, Ki=0.001, Kd=60, sign=1)
R_controller = PID(Kp=30, Ki=0.001, Kd=5, sign=-1)
#R_controller = PID(Kp=100, Ki=0.8, Kd=100, sign=-1) 无前馈最佳参数
est = make_radius_estimator(10)
est_circles = []
β_refs = []
R_refs = []
l1ac = L1AC_Struct()

ω0 = 20
ω_ff = 0

est_circle = update!(est, car.state.x, car.state.y)

cx = 0.
cy = 0.
theta = - π
RR = 15
Cx = cx + RR * cos(theta)
Cy = cy + RR * sin(theta)

texture = "snow"

for i in 1:total_steps

    if texture == "snow"
        β_ff, δ_ff = β_ff_snow, δ_ff_snow
        car.tyre = Tyre(B=5, C=2, D=0.3)
        ω_ff = 32.27746
    else
        β_ff, δ_ff = β_ff_ice, δ_ff_ice
        car.tyre = Tyre(B=4, C=2, D=0.15)
        ω_ff = 24.79148
    end

    if i % 1000 == 0
        theta += π / 36
        Cx = cx + RR * cos(theta)
        Cy = cy + RR * sin(theta)
    end

    est_circle = update!(est, car.state.x, car.state.y)
    R = est_circle.r
    
    s = car.state
    
    β = atan(s.ẏ, s.ẋ) - s.ψ
    err = round_angle(β - β_ref)
    δ = clamp(control!(β_controller, err), -π / 8, π / 8)
    
    if i % 10 == 0
        
        angle1 = atan(Cy - s.y, Cx - s.x)
        angle2 = atan(s.ẏ, s.ẋ)
        dangle = round_angle(angle1 - angle2)
        dis = sqrt((s.x - Cx) ^ 2 + (s.y - Cy) ^ 2)
        dR = R0 / 2 * tanh(dis) * cos(dangle)

        R_ref = R0 + dR
        if i > 100
            R_l1ac = L1AC!(l1ac, convert(Float64,R_ref), est_circle.r, 0.10)
            err = R - R_l1ac
            ω0 = clamp(ω_ff + control!(R_controller, err), 15, 45)
        else
            err = R - R_ref
            ω0 = clamp(ω_ff + control!(R_controller, err), 15, 45)
        end
    end
    ω = ω0
    # apply control and save data
    u = Input(δ, ω, ω)
    data_point = DriftData(s, u, compute_extended_state(s, u, car.body, car.tyre), R)
    step!(car, u, dt=0.01)
    
    if i < total_steps / 3
        texture = "snow"
    elseif i < total_steps * 2 / 3
        texture = "snow"
    else
        texture = "snow"
    end

    push!(data,  data_point)
    push!(β_refs, β_ref)
    push!(R_refs, R_ref)
    push!(est_circles, est_circle)
end

In [ ]:
total_steps = 40000

R0 = 10
β0 = - π / 2 * (3/4)
R_ref = R0
β_ref = - π / 2 * (3/4)
R_l1ac = R0

data = []

car = Car(state=State(x=-1., y=-11., ψ = 0., ẋ = 1., ẏ = 0., ψ̇ = 0.),tyre=Tyre(B=6, C=2, D=0.3))
β_controller = PID(Kp=4, Ki=0.001, Kd=60, sign=1)
R_controller = PID(Kp=30, Ki=0.001, Kd=5, sign=-1)
#R_controller = PID(Kp=100, Ki=0.8, Kd=100, sign=-1) 无前馈最佳参数
est = make_radius_estimator(10)
est_circles = []
β_refs = []
R_refs = []
l1ac = L1AC_Struct()

ω0 = 20
ω_ff = 0

est_circle = update!(est, car.state.x, car.state.y)

Cx = 0.
Cy = 0.

texture = "snow"

for i in 1:total_steps

    if texture == "snow"
        β_ff, δ_ff = β_ff_snow, δ_ff_snow
        car.tyre = Tyre(B=5, C=2, D=0.3)
        ω_ff = 32.27746
    else
        β_ff, δ_ff = β_ff_ice, δ_ff_ice
        car.tyre = Tyre(B=4, C=2, D=0.15)
        ω_ff = 24.79148
    end

    est_circle = update!(est, car.state.x, car.state.y)
    R = est_circle.r
    
    s = car.state
    
    β = atan(s.ẏ, s.ẋ) - s.ψ
    err = round_angle(β - β_ref)
    δ = clamp(control!(β_controller, err), -π / 8, π / 8)
    
    if i % 10 == 0
        
        angle1 = atan(Cy - s.y, Cx - s.x)
        angle2 = atan(s.ẏ, s.ẋ)
        dangle = round_angle(angle1 - angle2)
        dis = sqrt((s.x - Cx) ^ 2 + (s.y - Cy) ^ 2)
        dR = 1/3 * R0 * tanh(dis) * cos(dangle)

        R_ref = R0 + dR
        if i > 100
            #R_l1ac = L1AC!(l1ac, convert(Float64,R_ref), est_circle.r, 0.10)
            err = R - R_ref
            ω0 = clamp(ω_ff + control!(R_controller, err), 15, 45)
        else
            err = R - R_ref
            ω0 = clamp(ω_ff + control!(R_controller, err), 15, 45)
        end
    end
    ω = ω0
    # apply control and save data
    u = Input(δ, ω, ω)
    data_point = DriftData(s, u, compute_extended_state(s, u, car.body, car.tyre), R)
    step!(car, u, dt=0.01)
    
    if i > total_steps / 2
        texture = "ice"
    else
        texture = "snow"
    end

    push!(data,  data_point)
    push!(β_refs, β_ref)
    push!(R_refs, R_ref)
    push!(est_circles, est_circle)
end